In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import time
import tensorflow as tf
import pandas as pd
import gensim


import p8_util
import p8_util_config
import p9_util


In [4]:
OUTPUT_DIR = './tmp/baseline'
OUTPUT_DIR_TB = './tmp'
is_tensorboard = True

#### Tensorboard processes are killed

In [5]:
# this will kill the processes for Tensorboard
#is_tensorboard = True
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

kill: (28097): Aucun processus de ce type
kill: (28103)

In [ ]:
import p5_util

filename = './data/X_train_encoded.dump'
x_train = p5_util.object_load(filename)

filename = './data/X_test_encoded.dump'
x_test = p5_util.object_load(filename)

filename = './data/y_test.dump'
y_test = p5_util.object_load(filename)

filename = './data/y_train.dump'
y_train = p5_util.object_load(filename)



In [ ]:
import numpy as np
print(x_train.shape, np.array(y_train).shape)

    From now, x_train.shape[1] is used to configure `p8_util_config.ADANET_FEATURE_SHAPE`

#### Building AdaNet builder

In [ ]:
import p8_util
print("Input output dir= {}".format(OUTPUT_DIR))

oNNAdaNetBuilder = p8_util.create_nn_builder(OUTPUT_DIR)
oNNAdaNetBuilder.show()
params = {'net_builder': oNNAdaNetBuilder}

#### Create a Baseline Estimator

In [6]:
import BaselineEstimator

oNNAdaNetBuilder = p8_util.create_nn_builder(OUTPUT_DIR)
oNNAdaNetBuilder.show()
params = {'net_builder': oNNAdaNetBuilder}
#oNNAdaNetBuilder = None
oBaselineEstimator = BaselineEstimator.BaselineEstimator( params, IS_DEBUG=True)


*** Number of convolutional layers= 1

*** NNAdaNetBuilder : NN Type=RNN

*** get_tf_head() : feature shape= (1, 37)

*** get_tf_head() : feature columns= [NumericColumn(key='texts', shape=(1, 37), default_value=None, dtype=tf.float32, normalizer_fn=None)]

*** make_config() : output dir= ./tmp/baseline/RNN/SGRU

Max steps= 10 / Number of EPOCH=3


Global steps  : ................................... 10
NN type              : ............................ RNN
Features shape       : ............................ (1, 37)
Adanet outputdir     : ............................ ./tmp/baseline/RNN
Adanet output log    : ............................ ./tmp/baseline/RNN/SGRU
Adanet boosting iter.: ............................ 40
Adanet iter per boost: ............................ 0
Number of layers     : ............................ 1
Dropout rate         : ............................ 0.0
Seed value           : ............................ 42
Nb of classes (logit): ............................ 1
Ad

I0702 09:48:17.323605 139667762362176 estimator.py:201] Using config: {'_model_dir': './tmp/baseline/RNN/SGRU', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0680152be0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [4]:
import p5_util

filename = './data/keras_tokenizer.dump'
keras_tokenizer = p5_util.object_load(filename)

vocab_size = len(keras_tokenizer.word_index) + 1

print("Vocabulary size= {}".format(vocab_size))

p5_util.object_load : fileName= ./data/keras_tokenizer.dump
Vocabulary size= 129260


In [15]:
#import tensorflow as tf
import keras

embed_dim = 128
lstm_out = 1
batch_size = 32

model =  keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, embed_dim, input_length = 37, dropout = 0.2))
#model.add(keras.layers.LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(keras.layers.LSTM(embed_dim))
model.add(keras.layers.Dense(1,activation='linear'))
model.compile(loss = 'mean_squared_error', optimizer='sgd',metrics = ['mae'])
print(model.summary())


/home/bangui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  if __name__ == '__main__':


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 37, 128)           16545280  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 16,676,993
Trainable params: 16,676,993
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
batch_size = x_train.shape[0]//100

NameError: name 'x_train' is not defined

In [ ]:
model.fit(x_train, y_train, batch_size =batch_size, nb_epoch = 1,  verbose = 5)

In [ ]:
results, start_time, end_time  = oBaselineEstimator.train_and_evaluate()

In [ ]:
print(type(oBaselineEstimator._features))
oBaselineEstimator._features

In [ ]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))
print("------------------------------------------------\n")


In [ ]:
output_dir_log = './tmp/baseline/RNN/RNN'

In [ ]:
oNNAdaNetBuilder.output_dir_log, is_tensorboard
output_dir_log = oNNAdaNetBuilder.output_dir_log
output_dir_log

In [ ]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6007 &'
        .format(output_dir_log)
    )


    get_ipython().system_raw('./assets/ngrok http 6007 &')

In [ ]:
is_tensorboard,output_dir_log